# 패키지 불러오기

In [1]:
import surprise
print(surprise.__version__)

from surprise import SVD, Reader, Dataset, accuracy, SVDpp ,NMF, KNNBasic, CoClustering, SlopeOne, KNNWithMeans,BaselineOnly, KNNBaseline
from surprise.model_selection import  cross_validate, GridSearchCV , train_test_split
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn
import numpy as np

1.1rc0


# 데이터 준비

In [2]:
ratings = pd.read_csv('transactions.csv',encoding='cp949')
ratings.head()

,uid,time,item
0,0,2000-06-25 12:12,에스티로더
1,0,2000-06-25 12:42,시슬리
2,0,2000-08-26 18:10,크리니크
3,0,2000-08-26 18:30,듀퐁
4,0,2000-09-03 18:02,랑콤


In [3]:
# Rating값을 만들기 위한 Pivot 작업 
a = ratings.pivot_table(index='uid',columns='item',values='time',aggfunc=np.size,fill_value=0).reset_index()
a.columns.name = None;a.head()

,uid,012베네통,1492,1492마일즈,96NY,A-AND,AB.F.Z,ABFZ,AEG,AQ넥타이,...,후부,후첸로이터,휀시구두,휘나래,휠라슈즈,휠라의류,휠라인티모,휠라키즈,휴고보스,흙침대
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Pivot한 DF를 Long Form 형태의 데이터로 변환 -> uid, item, rating에 대한 정보를 가지도록 
ratings = pd.melt(a,id_vars='uid',value_name='rating',var_name='item')
ratings.shape

(1338000, 3)

In [5]:
# rating 값을 0과 1의 Sclae 가지도록 변환 -> 구매여부 데이터로 만듦.
ratings.rating.unique()

array([  0,   1,   6,   4,   3,   2,   5,   8,  14,   7,  15,  11,  12,
        10,  20,   9,  16,  34,  13,  19,  21,  17,  40,  55,  50,  33,
        30,  36,  18,  79,  39,  99,  37,  48,  27,  22,  45, 147,  54,
        64,  24,  29,  23,  41,  53,  31, 101, 207,  47, 111,  25,  35,
        38,  46,  43,  71,  32,  80,  57,  28, 138,  83,  49,  61,  52,
        42,  70,  86,  59], dtype=int64)

In [6]:
ratings.head(3)

,uid,item,rating
0,0,012베네통,0
1,1,012베네통,0
2,2,012베네통,0


In [7]:
# ratings = ratings.sort_values('uid')
# ratings.index = range(len(ratings))
ratings['rating'] = ratings['rating'].map(lambda x : 0 if x == 0 else 1  ) + 1
ratings.head()

,uid,item,rating
0,0,012베네통,1
1,1,012베네통,1
2,2,012베네통,1
3,3,012베네통,1
4,4,012베네통,1


In [8]:
ratings.rating.unique()

array([1, 2], dtype=int64)

In [9]:
# Surprise에 삽입하기 전 Surprise에서 원하는 형태로 데이터를 지정 
# 이때 scale이 1,2인 이유는 NMF를 실행할 떄 ZeroDivision 문제가 발생해 이를 없애기 위해 rating에 +1을 해주었음 
reader = Reader(rating_scale = (1,2))
data_set = Dataset.load_from_df(ratings[['uid','item','rating']],reader)

## 앞으로 해야할일 

* model tunning. 

* cross validate 성능. 

* precision, recall 값 확인하기. 


In [10]:
# SVD, Reader, Dataset, accuracy, SVDpp ,NMF, KNNBasic, KNNBaseline, CoClustering, SlopeOne, KNNWithMeans, KNNWithZScore
model = [SVD ,SVDpp ,NMF ,KNNBasic ,
        KNNBaseline , KNNWithMeans,
        CoClustering , SlopeOne, BaselineOnly]

In [11]:
grid_svd =  {'n_epochs': [20, 40], 'n_factors': [50, 100], 'lr_all':[0.001,0.01,0.1],'random_state':[0] }
grid_svdpp =  {'n_epochs': [20,40], 'n_factors': [50,100],'random_state':[0] }
grid_nmf =  {'n_epochs': [20, 40, 50], 'n_factors': [15, 30],'random_state':[0]}
grid_basic = {'k':[20,40,60],'min_k':[1,3],'random_state':[0]}
grid_base =  {'k':[20,40,60],'min_k':[1,3],'random_state':[0]}
grid_means =  {'k':[20,40,60],'min_k':[1,3],'random_state':[0]}
grid_Co = {'n_cltr_u':[2,3,5],'n_cltr_i':[2,3,5],'n_epochs':[10,20],'random_state':[0]}
# grid_slop = slope는 모르겠다.  

In [12]:
grid_svd_t = GridSearchCV(model[0], grid_svd, measures=['rmse', 'mae'], cv=3)
grid_svd_t.fit(data_set)
print(grid_svd_t.best_score['rmse'])
print(grid_svd_t.best_params['rmse'])

0.1087806182792397
{'n_epochs': 20, 'n_factors': 100, 'lr_all': 0.01, 'random_state': 0}


In [ ]:
# 도저히 돌릴 엄두가 나지 않습니다. 교수님... 
grid_svdpp_t = GridSearchCV(model[1], grid_svdpp, measures=['rmse', 'mae'], cv=3)
grid_svdpp_t.fit(data_set)
print(grid_svdpp_t.best_score['rmse'])
print(grid_svdpp_t.best_params['rmse'])

In [63]:
grid_nmf_t = GridSearchCV(model[2], grid_nmf, measures=['rmse', 'mae'], cv=3)
grid_nmf_t.fit(data_set)
print(grid_nmf_t.best_score['rmse'])
print(grid_nmf_t.best_params['rmse'])

0.10933546181627735
{'n_epochs': 20, 'n_factors': 15, 'random_state': 0}


In [62]:
grid_basic_t = GridSearchCV(model[3], grid_basic, measures=['rmse', 'mae'], cv=3)
grid_basic_t.fit(data_set)
print(grid_basic_t.best_score['rmse'])
print(grid_basic_t.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [64]:
grid_base_t = GridSearchCV(model[4], grid_base, measures=['rmse', 'mae'], cv=3)
grid_base_t.fit(data_set)
print(grid_base_t.best_score['rmse'])
print(grid_base_t.best_params['rmse'])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [65]:
grid_means_t = GridSearchCV(model[5], grid_means, measures=['rmse', 'mae'], cv=3)
grid_means_t.fit(data_set)
print(grid_means_t.best_score['rmse'])
print(grid_means_t.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [66]:
grid_Co_t = GridSearchCV(model[6], grid_Co, measures=['rmse', 'mae'], cv=3)
grid_Co_t.fit(data_set)
print(grid_Co_t.best_score['rmse'])
print(grid_Co_t.best_params['rmse'])

0.10808746236751765
{'n_cltr_u': 5, 'n_cltr_i': 2, 'n_epochs': 20, 'random_state': 0}


## 튜닝 후 모델 Crossvalid 

In [68]:
model_svd_tune = SVD(n_epochs =  20, n_factors =  100, lr_all=  0.01, random_state =  0)
cross_validate(model_svd_tune, data_set, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1083  0.1094  0.1076  0.1098  0.1088  0.1088  0.0008  
MAE (testset)     0.0255  0.0259  0.0255  0.0260  0.0261  0.0258  0.0003  
Fit time          70.10   70.22   68.88   68.68   70.03   69.58   0.66    
Test time         3.80    3.74    3.83    3.93    3.91    3.84    0.07    


{'test_rmse': array([0.10831665, 0.10944864, 0.10756789, 0.10979617, 0.10880702]),
 'test_mae': array([0.02551811, 0.0259116 , 0.02547544, 0.02596267, 0.0261012 ]),
 'fit_time': (70.0985517501831,
  70.21830606460571,
  68.87685441970825,
  68.68437004089355,
  70.02674555778503),
 'test_time': (3.801835775375366,
  3.742994546890259,
  3.8277621269226074,
  3.9294962882995605,
  3.9055960178375244)}

In [16]:
import time
startTime = time.time()

svdpp = SVDpp(random_state=0)
cross_validate(svdpp, data_set, measures=['RMSE', 'MAE'], cv=5, verbose=True)

endTime = time.time() - startTime
print(endTime)
# cross valid 94800초... 

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1077  0.1099  0.1082  0.1083  0.1090  0.1086  0.0008  
MAE (testset)     0.0253  0.0259  0.0255  0.0255  0.0257  0.0256  0.0002  
Fit time          17824.0319372.0419505.5417497.1618666.4418573.04804.39  
Test time         392.77  382.46  308.60  324.59  523.92  386.47  75.95   
94809.20407176018


In [69]:
model_nmf_tune = NMF(n_epochs= 20, n_factors= 15, random_state= 0)
cross_validate(model_nmf_tune, data_set, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1104  0.1094  0.1078  0.1094  0.1097  0.1094  0.0008  
MAE (testset)     0.0301  0.0301  0.0298  0.0300  0.0301  0.0300  0.0001  
Fit time          30.22   30.71   31.15   30.86   30.46   30.68   0.32    
Test time         3.66    3.82    3.70    3.60    3.59    3.67    0.08    


{'test_rmse': array([0.11038397, 0.10943183, 0.10782085, 0.10943391, 0.10973727]),
 'test_mae': array([0.03013064, 0.03008718, 0.02981502, 0.03000056, 0.03006831]),
 'fit_time': (30.22419309616089,
  30.713884830474854,
  31.154707193374634,
  30.86149263381958,
  30.456576108932495),
 'test_time': (3.6611969470977783,
  3.8227267265319824,
  3.695162296295166,
  3.599336624145508,
  3.5883681774139404)}

In [15]:
model_basic_tune = KNNBasic(k = 60, min_k = 1, random_state = 0)
cross_validate(model_basic_tune, data_set, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1099  0.1100  0.1096  0.1112  0.1110  0.1103  0.0006  
MAE (testset)     0.0154  0.0153  0.0151  0.0155  0.0155  0.0154  0.0001  
Fit time          24.64   24.36   24.76   24.32   24.29   24.48   0.19    
Test time         186.71  160.07  169.06  184.87  180.69  176.28  10.17   


{'test_rmse': array([0.10990461, 0.1099856 , 0.10959249, 0.111163  , 0.11095615]),
 'test_mae': array([0.01536798, 0.01528523, 0.01511461, 0.01551994, 0.01549038]),
 'fit_time': (24.64302110671997,
  24.36378240585327,
  24.760759353637695,
  24.32093572616577,
  24.289018869400024),
 'test_time': (186.71180486679077,
  160.06844425201416,
  169.0589861869812,
  184.86823081970215,
  180.6891574859619)}

In [16]:
model_base_tune = KNNBaseline(k = 60, min_k = 1, random_state = 0)
cross_validate(model_base_tune, data_set, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1107  0.1090  0.1086  0.1100  0.1095  0.1095  0.0007  
MAE (testset)     0.0254  0.0251  0.0250  0.0252  0.0251  0.0252  0.0001  
Fit time          27.84   27.85   28.07   28.04   28.04   27.97   0.10    
Test time         190.53  204.99  200.18  179.70  176.68  190.42  11.06   


{'test_rmse': array([0.11067051, 0.10902083, 0.10858495, 0.10996179, 0.10949841]),
 'test_mae': array([0.02535736, 0.02509733, 0.02504003, 0.0251994 , 0.02507364]),
 'fit_time': (27.843512773513794,
  27.848638772964478,
  28.07289719581604,
  28.043243646621704,
  28.040338277816772),
 'test_time': (190.52637696266174,
  204.99256348609924,
  200.18286442756653,
  179.70141506195068,
  176.6778793334961)}

In [17]:
model_mean_tune = KNNWithMeans(k = 60, min_k = 1, random_state = 0)
cross_validate(model_mean_tune, data_set, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1096  0.1081  0.1093  0.1095  0.1112  0.1096  0.0010  
MAE (testset)     0.0254  0.0251  0.0254  0.0253  0.0256  0.0254  0.0002  
Fit time          24.39   24.65   24.85   24.69   24.71   24.66   0.15    
Test time         164.86  178.27  181.65  173.18  173.04  174.20  5.69    


{'test_rmse': array([0.10961098, 0.10811974, 0.10934424, 0.10953268, 0.11119409]),
 'test_mae': array([0.02537351, 0.02513203, 0.02537687, 0.02531126, 0.02564145]),
 'fit_time': (24.391693115234375,
  24.648043155670166,
  24.854537963867188,
  24.690675497055054,
  24.709857940673828),
 'test_time': (164.86295175552368,
  178.2682011127472,
  181.65304851531982,
  173.18196773529053,
  173.03623175621033)}

In [18]:
model_Co_tune = CoClustering(n_cltr_u= 5, n_cltr_i= 2, n_epochs= 20, random_state= 0)
cross_validate(model_Co_tune, data_set, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1083  0.1084  0.1082  0.1075  0.1076  0.1080  0.0004  
MAE (testset)     0.0243  0.0244  0.0243  0.0241  0.0241  0.0242  0.0001  
Fit time          30.46   30.79   30.94   30.93   31.08   30.84   0.21    
Test time         2.22    2.22    2.76    1.71    2.76    2.33    0.40    


{'test_rmse': array([0.10833752, 0.10840562, 0.10817118, 0.10754992, 0.10762276]),
 'test_mae': array([0.02427153, 0.02436943, 0.02425058, 0.02406766, 0.02411071]),
 'fit_time': (30.459043979644775,
  30.79062795639038,
  30.939228534698486,
  30.925268173217773,
  31.077919960021973),
 'test_time': (2.2210581302642822,
  2.215073585510254,
  2.7576825618743896,
  1.7084832191467285,
  2.762547492980957)}

In [19]:
model_slope = SlopeOne()
cross_validate(model_slope, data_set, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1076  0.1090  0.1087  0.1083  0.1085  0.1084  0.0005  
MAE (testset)     0.0248  0.0251  0.0250  0.0250  0.0250  0.0250  0.0001  
Fit time          86.52   94.48   94.66   92.91   94.86   92.69   3.16    
Test time         225.48  221.99  227.58  225.65  209.70  222.08  6.45    


{'test_rmse': array([0.10760215, 0.1089728 , 0.10868311, 0.10831098, 0.10847233]),
 'test_mae': array([0.02482256, 0.02513277, 0.02498213, 0.02495352, 0.02502482]),
 'fit_time': (86.5235288143158,
  94.47721338272095,
  94.66081190109253,
  92.91421818733215,
  94.86434388160706),
 'test_time': (225.47584867477417,
  221.98713207244873,
  227.58317732810974,
  225.64933252334595,
  209.70272541046143)}

In [20]:
model_base = BaselineOnly()
cross_validate(model_base, data_set, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1087  0.1078  0.1094  0.1089  0.1071  0.1084  0.0008  
MAE (testset)     0.0250  0.0247  0.0251  0.0251  0.0248  0.0249  0.0002  
Fit time          3.65    4.03    4.06    4.04    4.05    3.97    0.16    
Test time         3.01    1.66    3.07    1.78    3.00    2.50    0.64    


{'test_rmse': array([0.10869986, 0.10784098, 0.10940705, 0.10893857, 0.10714275]),
 'test_mae': array([0.02499209, 0.02467891, 0.02511214, 0.02509441, 0.02479735]),
 'fit_time': (3.654224157333374,
  4.0292582511901855,
  4.059196710586548,
  4.036203384399414,
  4.049168348312378),
 'test_time': (3.005958080291748,
  1.6585779190063477,
  3.0707285404205322,
  1.7782411575317383,
  2.9950437545776367)}

### recall and pricision score 

In [10]:
from collections import defaultdict

from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [21]:
kf = KFold(n_splits=5,random_state=0)
model_svd_tune = SVD(n_epochs =  20, n_factors =  100, lr_all=  0.01, random_state =  0)
precision_svd = []
recall_svd = []
i = 1
for trainset, testset in kf.split(data_set):
    model_svd_tune.fit(trainset)
    predictions = model_svd_tune.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=1.5)
    p = sum(prec for prec in precisions.values()) / len(precisions)
    r = sum(rec for rec in recalls.values()) / len(recalls)
    
    precision_svd.append(p)
    recall_svd.append(r)
    
    # Precision and recall can then be averaged over all users
    print('iter {} precisions:'.format(i),p)
    print('iter {} recalls:'.format(i),r)
    i += 1
print('Mean Precision Score is ', np.mean(precision_svd))
print('std Precision Score is ', np.std(precision_svd))
print('Mean recall Score is ', np.mean(recall_svd))
print('std recall Score is ', np.std(recall_svd))

iter 1 precisions: 0.924
iter 1 recalls: 0.20196911976911971
iter 2 precisions: 0.931
iter 2 recalls: 0.2056928926465692
iter 3 precisions: 0.932
iter 3 recalls: 0.20524818139376963
iter 4 precisions: 0.946
iter 4 recalls: 0.17738269508269497
iter 5 precisions: 0.917
iter 5 recalls: 0.18542189697557332
Mean Precision Score is  0.9299999999999999
std Precision Score is  0.009654014708917708
Mean recall Score is  0.19514295717354538
std recall Score is  0.011575310896174548


In [22]:
# NMF
kf = KFold(n_splits=5,random_state=0)
model_nmf_tune =NMF(n_epochs= 20, n_factors= 15, random_state= 0)
precision_nmf = []
recall_nmf = []
i = 1
for trainset, testset in kf.split(data_set):
    model_nmf_tune.fit(trainset)
    predictions = model_nmf_tune.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=1.5)
    p = sum(prec for prec in precisions.values()) / len(precisions)
    r = sum(rec for rec in recalls.values()) / len(recalls)
    
    precision_nmf.append(p)
    recall_nmf.append(r)
    
    # Precision and recall can then be averaged over all users
    print('iter {} precisions:'.format(i),p)
    print('iter {} recalls:'.format(i),r)
    i += 1
print('Mean Precision Score is ', np.mean(precision_nmf))
print('std Precision Score is ', np.std(precision_nmf))
print('Mean recall Score is ', np.mean(recall_nmf))
print('std recall Score is ', np.std(recall_nmf))

iter 1 precisions: 0.931
iter 1 recalls: 0.20316656763715582
iter 2 precisions: 0.929
iter 2 recalls: 0.18425639262698093
iter 3 precisions: 0.93
iter 3 recalls: 0.19362829392829398
iter 4 precisions: 0.92
iter 4 recalls: 0.19555034214151853
iter 5 precisions: 0.94
iter 5 recalls: 0.18752153622521273
Mean Precision Score is  0.93
std Precision Score is  0.006356099432828252
Mean recall Score is  0.1928246265118324
std recall Score is  0.006581315244590704


In [23]:
# KNNbasic
kf = KFold(n_splits=5,random_state=0)
model_knnbasic_tune =KNNBasic(k = 60, min_k = 1, random_state = 0)
precision_knnbasic = []
recall_knnbasic = []
i = 1
for trainset, testset in kf.split(data_set):
    model_knnbasic_tune.fit(trainset)
    predictions = model_knnbasic_tune.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=1.5)
    p = sum(prec for prec in precisions.values()) / len(precisions)
    r = sum(rec for rec in recalls.values()) / len(recalls)
    
    precision_knnbasic.append(p)
    recall_knnbasic.append(r)
    
    # Precision and recall can then be averaged over all users
    print('iter {} precisions:'.format(i),p)
    print('iter {} recalls:'.format(i),r)
    i += 1
print('Mean Precision Score is ', np.mean(precision_knnbasic))
print('std Precision Score is ', np.std(precision_knnbasic))
print('Mean recall Score is ', np.mean(recall_knnbasic))
print('std recall Score is ', np.std(recall_knnbasic))

Computing the msd similarity matrix...
Done computing similarity matrix.
iter 1 precisions: 1.0
iter 1 recalls: 0.167
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 2 precisions: 1.0
iter 2 recalls: 0.152
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 3 precisions: 1.0
iter 3 recalls: 0.142
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 4 precisions: 1.0
iter 4 recalls: 0.15220238095238095
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 5 precisions: 1.0
iter 5 recalls: 0.15
Mean Precision Score is  1.0
std Precision Score is  0.0
Mean recall Score is  0.1526404761904762
std recall Score is  0.00808690187027211


In [24]:
# KNNBaseline
kf = KFold(n_splits=5,random_state=0)
model_KNNBaseline_tune =KNNBaseline(k = 60, min_k = 1, random_state = 0)
precision_KNNBaseline = []
recall_KNNBaseline = []
i = 1
for trainset, testset in kf.split(data_set):
    model_KNNBaseline_tune.fit(trainset)
    predictions = model_KNNBaseline_tune.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=1.5)
    p = sum(prec for prec in precisions.values()) / len(precisions)
    r = sum(rec for rec in recalls.values()) / len(recalls)
    
    precision_KNNBaseline.append(p)
    recall_KNNBaseline.append(r)
    
    # Precision and recall can then be averaged over all users
    print('iter {} precisions:'.format(i),p)
    print('iter {} recalls:'.format(i),r)
    i += 1
print('Mean Precision Score is ', np.mean(precision_KNNBaseline))
print('std Precision Score is ', np.std(precision_KNNBaseline))
print('Mean recall Score is ', np.mean(recall_KNNBaseline))
print('std recall Score is ', np.std(recall_KNNBaseline))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 1 precisions: 1.0
iter 1 recalls: 0.15314583333333331
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 2 precisions: 1.0
iter 2 recalls: 0.165
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 3 precisions: 1.0
iter 3 recalls: 0.1410769230769231
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 4 precisions: 1.0
iter 4 recalls: 0.16817424242424242
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 5 precisions: 1.0
iter 5 recalls: 0.16543611111111114
Mean Precision Score is  1.0
std Precision Score is  0.0
Mean recall Score is  0.158566621989122
std recall Score is  0.010160306309786316


In [25]:
# KNNWithmeanscore
kf = KFold(n_splits=5,random_state=0)
model_KNNMEAN_tune =KNNWithMeans(k = 60, min_k = 1, random_state = 0)
precision_KNNMEAN = []
recall_KNNMEAN = []
i = 1
for trainset, testset in kf.split(data_set):
    model_KNNMEAN_tune.fit(trainset)
    predictions = model_KNNMEAN_tune.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=1.5)
    p = sum(prec for prec in precisions.values()) / len(precisions)
    r = sum(rec for rec in recalls.values()) / len(recalls)
    
    precision_KNNMEAN.append(p)
    recall_KNNMEAN.append(r)
    
    # Precision and recall can then be averaged over all users
    print('iter {} precisions:'.format(i),p)
    print('iter {} recalls:'.format(i),r)
    i += 1
print('Mean Precision Score is ', np.mean(precision_KNNMEAN))
print('std Precision Score is ', np.std(precision_KNNMEAN))
print('Mean recall Score is ', np.mean(recall_KNNMEAN))
print('std recall Score is ', np.std(recall_KNNMEAN))

Computing the msd similarity matrix...
Done computing similarity matrix.
iter 1 precisions: 1.0
iter 1 recalls: 0.1593921568627451
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 2 precisions: 1.0
iter 2 recalls: 0.159
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 3 precisions: 1.0
iter 3 recalls: 0.1511969696969697
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 4 precisions: 1.0
iter 4 recalls: 0.14918382352941176
Computing the msd similarity matrix...
Done computing similarity matrix.
iter 5 precisions: 1.0
iter 5 recalls: 0.16455882352941176
Mean Precision Score is  1.0
std Precision Score is  0.0
Mean recall Score is  0.15666635472370766
std recall Score is  0.00567572739074159


In [26]:
# Coclustering
kf = KFold(n_splits=5,random_state=0)
model_Coclustering_tune = CoClustering(n_cltr_u= 5, n_cltr_i= 2, n_epochs= 20, random_state= 0)
precision_Coclustering = []
recall_Coclustering = []
i = 1
for trainset, testset in kf.split(data_set):
    model_Coclustering_tune.fit(trainset)
    predictions = model_Coclustering_tune.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=1.5)
    p = sum(prec for prec in precisions.values()) / len(precisions)
    r = sum(rec for rec in recalls.values()) / len(recalls)
    
    precision_Coclustering.append(p)
    recall_Coclustering.append(r)
    
    # Precision and recall can then be averaged over all users
    print('iter {} precisions:'.format(i),p)
    print('iter {} recalls:'.format(i),r)
    i += 1
print('Mean Precision Score is ', np.mean(precision_Coclustering))
print('std Precision Score is ', np.std(precision_Coclustering))
print('Mean recall Score is ', np.mean(recall_Coclustering))
print('std recall Score is ', np.std(recall_Coclustering))

iter 1 precisions: 0.93
iter 1 recalls: 0.1986975441225441
iter 2 precisions: 0.912
iter 2 recalls: 0.2242332763712559
iter 3 precisions: 0.939
iter 3 recalls: 0.19411113875804273
iter 4 precisions: 0.938
iter 4 recalls: 0.17801841304120708
iter 5 precisions: 0.931
iter 5 recalls: 0.2001919164169164
Mean Precision Score is  0.93
std Precision Score is  0.009695359714832628
Mean recall Score is  0.19905045774199323
std recall Score is  0.01484820262769046


In [27]:
# Slope
kf = KFold(n_splits=5,random_state=0)
model_Slope_tune = SlopeOne()
precision_Slope = []
recall_Slope = []
i = 1
for trainset, testset in kf.split(data_set):
    model_Slope_tune.fit(trainset)
    predictions = model_Slope_tune.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=1.5)
    p = sum(prec for prec in precisions.values()) / len(precisions)
    r = sum(rec for rec in recalls.values()) / len(recalls)
    
    precision_Slope.append(p)
    recall_Slope.append(r)
    
    # Precision and recall can then be averaged over all users
    print('iter {} precisions:'.format(i),p)
    print('iter {} recalls:'.format(i),r)
    i += 1
print('Mean Precision Score is ', np.mean(precision_Slope))
print('std Precision Score is ', np.std(precision_Slope))
print('Mean recall Score is ', np.mean(recall_Slope))
print('std recall Score is ', np.std(recall_Slope))

iter 1 precisions: 0.921
iter 1 recalls: 0.18974592896783288
iter 2 precisions: 0.941
iter 2 recalls: 0.19075420592479408
iter 3 precisions: 0.926
iter 3 recalls: 0.21312556650865463
iter 4 precisions: 0.934
iter 4 recalls: 0.19245734217473348
iter 5 precisions: 0.928
iter 5 recalls: 0.19958594183594183
Mean Precision Score is  0.93
std Precision Score is  0.006899275324264107
Mean recall Score is  0.19713379708239137
std recall Score is  0.008705156085845843


In [28]:
# Baselineonly
kf = KFold(n_splits=5,random_state=0)
model_Baselineonly_tune = BaselineOnly()
precision_Baselineonly = []
recall_Baselineonly = []
i = 1
for trainset, testset in kf.split(data_set):
    model_Baselineonly_tune.fit(trainset)
    predictions = model_Baselineonly_tune.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=1.5)
    p = sum(prec for prec in precisions.values()) / len(precisions)
    r = sum(rec for rec in recalls.values()) / len(recalls)
    
    precision_Baselineonly.append(p)
    recall_Baselineonly.append(r)
    
    # Precision and recall can then be averaged over all users
    print('iter {} precisions:'.format(i),p)
    print('iter {} recalls:'.format(i),r)
    i += 1
print('Mean Precision Score is ', np.mean(precision_Baselineonly))
print('std Precision Score is ', np.std(precision_Baselineonly))
print('Mean recall Score is ', np.mean(recall_Baselineonly))
print('std recall Score is ', np.std(recall_Baselineonly))

Estimating biases using als...
iter 1 precisions: 0.929
iter 1 recalls: 0.19042942839324412
Estimating biases using als...
iter 2 precisions: 0.918
iter 2 recalls: 0.18465017662076474
Estimating biases using als...
iter 3 precisions: 0.921
iter 3 recalls: 0.19781277056277055
Estimating biases using als...
iter 4 precisions: 0.944
iter 4 recalls: 0.19775796536796533
Estimating biases using als...
iter 5 precisions: 0.938
iter 5 recalls: 0.20115258352758347
Mean Precision Score is  0.9299999999999999
std Precision Score is  0.00985900603509295
Mean recall Score is  0.19436058489446564
std recall Score is  0.005988940568773352
